In [1]:
from sqlalchemy import create_engine
import xmlrpc.client
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE_CONNECTION_URI = os.environ["DB_URL"]
ODOO_URL = os.environ["ODOO_URL"]
ODOO_NAME = os.environ["ODOO_NAME"]
ODOO_USERNAME = os.environ["ODOO_USERNAME"]
ODOO_PASSWORD = os.environ["ODOO_PASSWORD"]

# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

class OdooAPI:
    def __init__(self):
        self.dbUrl = ODOO_URL
        self.dbName = ODOO_NAME
        self.dbUsername = ODOO_USERNAME
        self.dbPassword = ODOO_PASSWORD
        self.common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(self.dbUrl))
        self.uid = self.common.authenticate(self.dbName, self.dbUsername, self.dbPassword, {})
        self.models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(self.dbUrl))
        self.country_codes = {
            "Argentina": "AR",
            "Paraguay": "PY",
            "Bolivia": "BO",
            "Uruguay": "UY",
            "Chile": "CL",
            "Perú": "PE",
            "Peru": "PE",
            "Ecuador": "EC",
            "Colombia": "CO",
            "Venezuela": "VE",
            "Brasil": "BR",
            "Panamá": "PA",
            "Panama": "PA",
            "Costa Rica": "CR",
            "Honduras": "HN",
            "El Salvador": "SV",
            "Guatemala": "GT",
            "México": "MX",
            "Mexico": "MX",
            "República Dominicana": "DM",
            "Republica Dominicana": "DM",
            "Rep Dom": "DM",
            "Puerto Rico": "PR",
            "España": "ES",
        }

    # Data Handling Functions
    def id_field_into_1value(self, jsonFile):
        for index, line in enumerate(jsonFile):
            for item in line:
                if type(line[item]) == list:
                    jsonFile[index][item] = jsonFile[index][item][1]

    # Export to CSV functions
    def raw_export_projects(self, properties_dict):
        projects = self.models.execute_kw(self.dbName,
                                          self.uid,
                                          self.dbPassword,
                                          'project.project',
                                          'search_read',
                                          [],
                                          {'fields': list(properties_dict.keys()), })
        self.id_field_into_1value(projects)
        df = pd.DataFrame(projects)

        # exporta los proyectos a un srchivo .csv
        df.to_csv("raw_odoo_projects.csv", encoding="latin-1", index=False, errors='ignore')
        
        # inserta la data resultado de la extraccion en una tabla de la DB postgre
        df.to_sql('odoo_projects', con=engine, if_exists='replace', index=False)

        # retornar la data para poder visualizarla en "Variables" - dev
        # return projects

    def raw_export_subscriptions(self, properties_dict):
        subscriptions = self.models.execute_kw(self.dbName,
                                               self.uid,
                                               self.dbPassword,
                                               'sale.subscription.report',
                                               'search_read',
                                               [],
                                               {'fields': list(properties_dict.keys())})
        self.id_field_into_1value(subscriptions)
        df = pd.DataFrame(subscriptions)
        df.to_csv("raw_odoo_subscriptions.csv", encoding="latin-1", index=False, errors='ignore')
        return subscriptions

    def raw_export_sales_lines(self, properties_dict):
        salesLines = self.models.execute_kw(self.dbName,
                                            self.uid,
                                            self.dbPassword,
                                            'sale.order.line',
                                            'search_read',
                                            [],
                                            {'fields': list(properties_dict.keys())})
        self.id_field_into_1value(salesLines)
        df = pd.DataFrame(salesLines)
        df.to_csv("raw_odoo_sales_lines.csv", encoding="latin-1", index=False, errors='ignore')

    def raw_export_entities(self, properties_dict):
        entities = self.models.execute_kw(self.dbName,
                                          self.uid,
                                          self.dbPassword,
                                          'res.partner',
                                          'search_read',
                                          [],
                                          {'fields': list(properties_dict.keys())})
        self.id_field_into_1value(entities)
        df = pd.DataFrame(entities)
        df.to_csv("raw_odoo_entities.csv", encoding="latin-1", index=False, errors='ignore')

    def raw_export_subscriptions_lines(self, properties_dict):
        subscriptions_lines = self.models.execute_kw(self.dbName,
                                          self.uid,
                                          self.dbPassword,
                                          'sale.subscription.line',
                                          'search_read',
                                          [],
                                          {'fields': list(properties_dict.keys())})
        self.id_field_into_1value(subscriptions_lines)
        df = pd.DataFrame(subscriptions_lines)
        df.to_csv("raw_odoo_subscriptions_lines.csv", encoding="latin-1", index=False, errors='ignore')

    def handle_raw_odoo(self, csv_file, properties_dict={}, values_dict={}, date_columns=[]):
        # Csv into Dataframe
        df = pd.read_csv(csv_file + ".csv", encoding="latin-1")
        # Rename Columns
        df.rename(columns=properties_dict, inplace=True)
        # Rename Values
        df = df.applymap(lambda x: str(x) if pd.notnull(x) else '')
        df.replace(values_dict, inplace=True)
        # Columns into lower case and underscore union
        columns = [column.lower() for column in df.columns]
        columns = ["_".join(column.split(" ")) for column in columns]
        df.columns = columns
        # Date Standarization
        date_columns = [char.lower() for char in date_columns]
        date_columns = ["_".join(char.split(" ")) for char in date_columns]
        for date_column in date_columns:
            df[date_column] = pd.to_datetime(df[df[date_column].notna()][date_column], errors='raise')
            df[date_column] = df[date_column].dt.strftime('%Y-%m-%d')
        df.to_csv(csv_file[4:] + ".csv", encoding="latin-1", index=False, errors='ignore')

properties_dict = { 
    # 'x_project_id_sale_order_count': 1,
    'name': 1,
    'x_studio_complejidad': 1,
    'stage_id': 1,   
    'x_studio_etapa': 1,    
    'x_studio_das_detenido': 1,
    #'display_name': 1,
    #'description': 1,
    #'date_start': 1,
    'partner_id': 1,
    'x_studio_cliente_final': 1,
    'x_studio_mail_de_focal_point': 1,
    'user_id': 1,
    'x_studio_comercial': 1,
    'x_studio_regin_comercial': 1,
    'x_studio_pas': 1,
    'company_id': 1,
    'create_date': 1,
    'x_studio_cambio_a_relevamiento_inicial': 1,
    'x_studio_cambio_a_despliegue_y_soporte_en_puesta_en_marcha': 1,
    'x_studio_cambio_a_configuracindesarrollofabricacin_de_equipos': 1,
    'x_studio_fecha_de_finalizacin': 1,
    'x_studio_cambio_a_pausados': 1,
    'sale_order_id': 1, 
    'x_studio_cliente': 1,
    'x_studio_proyecto': 1,
    'x_studio_piloto': 1,
    'create_uid': 1
    #'x_studio_related_field_Wbiaw': 1,
}

odoo_api_instance = OdooAPI()

projects = odoo_api_instance.raw_export_projects(properties_dict)

os.remove("raw_odoo_projects" + ".csv")